In [19]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D , MaxPooling2D , Flatten , Dropout
import os
import pandas as pd
import numpy as np

In [20]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'

In [21]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        label_path = os.path.join(dir, label)
        if os.path.isdir(label_path):  # Check if it's a directory
            for imagename in os.listdir(label_path):
                image_path = os.path.join(label_path, imagename)
                if os.path.isfile(image_path):  # Check if it's a file (an image)
                    image_paths.append(image_path)
                    labels.append(label)
            print(label, "completed")
    return image_paths, labels

In [22]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

happy completed
sad completed
fear completed
surprise completed
neutral completed
angry completed
disgust completed


In [23]:
print(train)

                                image    label
0         images/train/happy/3578.jpg    happy
1        images/train/happy/16988.jpg    happy
2         images/train/happy/2666.jpg    happy
3         images/train/happy/5109.jpg    happy
4        images/train/happy/11981.jpg    happy
...                               ...      ...
28816  images/train/disgust/10112.jpg  disgust
28817  images/train/disgust/21668.jpg  disgust
28818   images/train/disgust/7049.jpg  disgust
28819   images/train/disgust/9716.jpg  disgust
28820   images/train/disgust/3561.jpg  disgust

[28821 rows x 2 columns]


In [25]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

happy completed
sad completed
fear completed
surprise completed
neutral completed
angry completed
disgust completed


In [26]:
print(test)

                                    image    label
0       images/validation/happy/23933.jpg    happy
1       images/validation/happy/24906.jpg    happy
2       images/validation/happy/18033.jpg    happy
3       images/validation/happy/15271.jpg    happy
4       images/validation/happy/26888.jpg    happy
...                                   ...      ...
7061  images/validation/disgust/20761.jpg  disgust
7062  images/validation/disgust/28710.jpg  disgust
7063  images/validation/disgust/23876.jpg  disgust
7064   images/validation/disgust/9460.jpg  disgust
7065  images/validation/disgust/35580.jpg  disgust

[7066 rows x 2 columns]


In [27]:
from tqdm.notebook import tqdm

In [30]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [31]:
train_features = extract_features(train['image']) 

  0%|          | 0/28821 [00:00<?, ?it/s]

/Users/rudra/anaconda3/lib/python3.11/site-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [32]:
test_features = extract_features(test['image']) 

  0%|          | 0/7066 [00:00<?, ?it/s]

In [33]:
x_train = train_features/255.0
x_test = test_features/255.0

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [35]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [36]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [37]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [38]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [41]:
model.fit(x=x_train,y=y_train,batch_size=128,epochs=25)

Epoch 1/25
226/226 [==============================] - 185s 818ms/step - loss: 1.7288 - accuracy: 0.2877
Epoch 2/25
226/226 [==============================] - 169s 748ms/step - loss: 1.6013 - accuracy: 0.3666
Epoch 3/25
226/226 [==============================] - 196s 868ms/step - loss: 1.5015 - accuracy: 0.4125
Epoch 4/25
226/226 [==============================] - 183s 809ms/step - loss: 1.4464 - accuracy: 0.4410
Epoch 5/25
226/226 [==============================] - 173s 763ms/step - loss: 1.4087 - accuracy: 0.4561
Epoch 6/25
226/226 [==============================] - 177s 784ms/step - loss: 1.3722 - accuracy: 0.4714
Epoch 7/25
226/226 [==============================] - 192s 848ms/step - loss: 1.3429 - accuracy: 0.4856
Epoch 8/25
226/226 [==============================] - 177s 784ms/step - loss: 1.3239 - accuracy: 0.4925
Epoch 9/25
226/226 [==============================] - 166s 736ms/step - loss: 1.3083 - accuracy: 0.5011
Epoch 10/25
226/226 [==============================] - 181s 802m

In [42]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

/Users/rudra/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
